In [2]:
!pip install line_profiler
import line_profiler
%load_ext line_profiler

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import pandas as pd
import string

N = 2_000_000
df = pd.DataFrame(np.random.randn(N, 4), columns=[f"col{i}" for i in range(4)])
df["key"] = np.random.choice(list(string.ascii_letters.lower()), N, replace=True)
df.head(2)

,col0,col1,col2,col3,key
0,0.283603,0.361292,0.821186,0.225738,d
1,1.079657,-0.072517,-0.438215,-0.779685,z


In [4]:
import numpy as np
import pandas as pd
from numba import jit, njit
import numba
from typing import Union

%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

In [9]:
recipes = pd.read_csv('recipes_sample (2).csv')
recipes.head(2)

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN


In [10]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
recipes.dtypes

name                      object
id                         int64
minutes                    int64
contributor_id             int64
submitted         datetime64[ns]
n_steps                  float64
description               object
n_ingredients            float64
dtype: object

In [13]:
reviews = pd.read_csv('reviews_sample (2).csv', index_col=0)
reviews.head(3)

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."


In [14]:
reviews['date'] = pd.to_datetime(reviews['date'])
reviews.dtypes

user_id               int64
recipe_id             int64
date         datetime64[ns]
rating                int64
review               object
dtype: object

## Измерение времени выполнения кода

Назовем полным описанием рецепта строку, полученную путем конкатенации названия и описания рецепта через пробел. Удалите строки для рецептов, которые были добавлены не в 2010 году.

Реализуйте несколько вариантов функции подсчета средней длины полного описания рецепта для рецептов, добавленных в 2010 году.

In [15]:
recipes['year'] = recipes['submitted'].dt.year
recipes.head(2)

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,year
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0,2002
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN,2003


In [16]:
recipes['year'] = recipes['submitted'].dt.year
recipes_new = recipes[recipes['year'] != 2010]

In [17]:
recipes_new.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,year
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0,2002
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN,2003
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0,2002
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN,2002
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN,2004


In [18]:
# reviews['year'] = reviews[reviews['date'].dt.year != 2010]
reviews['year'] = reviews['date'].dt.year
reviews_new = reviews[reviews['year'] != 2010]
reviews_new

,user_id,recipe_id,date,rating,review,year
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...,2003
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...,2007
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy...",2008
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...,2017
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...,2008
...,...,...,...,...,...,...
344676,724631,314698,2008-10-14,5,"5 Stars, The Hunter said the mushrooms and sau...",2008
1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...,2009
158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...,2012
1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou...",2008


In [19]:

recipes_new['concatination'] = recipes_new['name'] + ' ' + recipes_new['description']


C:\Users\Владимир\AppData\Local\Temp\ipykernel_14340\3697113525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipes_new['concatination'] = recipes_new['name'] + ' ' + recipes_new['description']


In [20]:
recipes_new

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,year,concatination
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0,2002,george s at the cove black bean soup an origi...
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN,2003,healthy for them yogurt popsicles my children...
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0,2002,"i can t believe it s spinach these were so go,..."
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN,2002,italian gut busters my sister-in-law made the...
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN,2004,love is in the air beef fondue sauces i thi...
...,...,...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0,2007,zurie s holey rustic olive and cheddar bread t...
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0,2009,zwetschgenkuchen bavarian plum cake this is a...
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN,2004,zwiebelkuchen southwest german onion cake th...
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN,2012,zydeco soup this is a delicious soup that i or...


In [21]:
recipes_2010 = recipes[recipes.submitted.dt.year == 2010]

In [22]:
recipes_2010['concatination'] = recipes_2010['name'] + ' ' + recipes_2010['description']
recipes_2010

C:\Users\Владимир\AppData\Local\Temp\ipykernel_14340\417134780.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipes_2010['concatination'] = recipes_2010['name'] + ' ' + recipes_2010['description']


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,year,concatination
52,just peachy cobbler,437637,70,1085867,2010-09-17,10.0,all i can say is yummmmmm . . . a simple to ma...,10.0,2010,just peachy cobbler all i can say is yummmmmm...
68,the heat spicy party mix,437219,95,1682162,2010-09-13,NaN,a spicy chex mix that will really warm your gu...,11.0,2010,the heat spicy party mix a spicy chex mix tha...
81,iowa state fair sweet dough caramel cinnamon ...,435816,80,17803,2010-08-24,29.0,this was the winning entry at the 2010 iowa st...,NaN,2010,iowa state fair sweet dough caramel cinnamon ...
104,1 minute blueberries cream,428566,2,1375473,2010-06-04,4.0,i was craving blueberry tonight but wanted non...,NaN,2010,1 minute blueberries cream i was craving blu...
146,2 2 2 diet mocha,416599,5,789314,2010-03-15,5.0,"while trying to come up with a satisfying ""sna...",7.0,2010,2 2 2 diet mocha while trying to come up with ...
...,...,...,...,...,...,...,...,...,...,...
29897,zoe s chicken tarragon,441211,40,76559,2010-11-04,12.0,from a good housekeeping at my hair salon. ha...,9.0,2010,zoe s chicken tarragon from a good housekeepin...
29907,zucchini and noodle slice,412518,60,423555,2010-02-10,21.0,"a yummy, tasty slice packed with vegies and ri...",13.0,2010,"zucchini and noodle slice a yummy, tasty slice..."
29915,zucchini bread bread machine,409757,220,539686,2010-01-22,7.0,"originally from a packet of red star yeast, th...",NaN,2010,zucchini bread bread machine originally from...
29926,zucchini chip cupcakes,406686,35,628076,2010-01-04,9.0,this is a great tasting recipe to use up zucch...,14.0,2010,zucchini chip cupcakes this is a great tasting...


Реализуйте несколько вариантов функции подсчета средней длины полного описания рецепта для рецептов, добавленных в 2010 году.

1\.1 С использованием метода `DataFrame.iterrows` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - нахождение полного описания рецепта осуществляется внутри цикла по `iterrows` для каждой строки по отдельности.

In [23]:
def get_mean_len_A(df: pd.DataFrame) -> float:
    cnt = 0
    a = 0
    for _, row in df.iterrows():
      cnt += len(row['concatination'].split())
      a += 1
    print(cnt/a)
%time get_mean_len_A(recipes_2010)

47.07867360208063
CPU times: total: 62.5 ms
Wall time: 45.2 ms


1\.2. С использованием метода `DataFrame.apply` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - вызываете метод apply у таблицы, в качестве аргумента передаете функцию, которая возвращает полное описание для каждой строки;
    
    - считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [24]:
def get_mean_len_B(df: pd.DataFrame) -> float:
  f = lambda row: len((row['name'] + ' ' + row['description']).split())
  df['concatination2'] = df.apply(f, axis = 1)
  print(df['concatination2'].mean())

%time get_mean_len_B(recipes_2010)

47.07867360208063
CPU times: total: 31.2 ms
Wall time: 32.4 ms


C:\Users\Владимир\AppData\Local\Temp\ipykernel_14340\2284781362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['concatination2'] = df.apply(f, axis = 1)


1\.3. С использованием векторизованных методов серий `pd.Series`:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - при помощи векторизированных операций получаете столбец с полным описанием;
    
    - при помощи векторизированных операций получаете длины полного описания;
    
    - при помощи метода серий получаете среднюю длину полных описаний. 

In [25]:
def get_mean_len_C(df: pd.DataFrame) -> float:
    df['concatination3'] = df['name'] + ' ' + df['description']
    print(df['concatination3'].str.split().str.len().mean())
%time get_mean_len_C(recipes_2010)

47.07867360208063
CPU times: total: 15.6 ms
Wall time: 21 ms


C:\Users\Владимир\AppData\Local\Temp\ipykernel_14340\2962223474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['concatination3'] = df['name'] + ' ' + df['description']


1.4 Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций при помощи магических команд `time` и `timeit`.

In [26]:
%time get_mean_len_A(recipes_2010)

47.07867360208063
CPU times: total: 46.9 ms
Wall time: 47 ms


In [27]:
%time get_mean_len_B(recipes_2010)

47.07867360208063
CPU times: total: 31.2 ms
Wall time: 33 ms


C:\Users\Владимир\AppData\Local\Temp\ipykernel_14340\2284781362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['concatination2'] = df.apply(f, axis = 1)


In [28]:
%time get_mean_len_C(recipes_2010)

47.07867360208063
CPU times: total: 31.2 ms
Wall time: 22 ms


C:\Users\Владимир\AppData\Local\Temp\ipykernel_14340\2962223474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['concatination3'] = df['name'] + ' ' + df['description']


## Анализ пошагового выполнения кода 

Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. 

In [29]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [30]:
import re


def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        recipe_id, review = row["recipe_id"], row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)

    word_reviews_count = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

2.1 Найдите узкие места в коде, проанализировав код функции по шагам, используя профайлер. Сохраните результаты работы профайлера в отдельную текстовую ячейку. Выпишите (словами), что в имеющемся коде реализовано неоптимально. 

In [31]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-06 s

Total time: 73.8365 s
File: <ipython-input-111-21baa068d50d>
Function: get_word_reviews_count at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     4                                           def get_word_reviews_count(df):
     5         1          3.0      3.0      0.0      word_reviews = {}
     6    126680   19679435.0    155.3     26.7      for _, row in df.dropna(subset=["review"]).iterrows():
     7    126679    3766688.0     29.7      5.1          recipe_id, review = row["recipe_id"], row["review"]
     8    126679    2289793.0     18.1      3.1          words = re.sub("[^A-Za-z\s]", "", review).split(" ")
     9   6918689    3314418.0      0.5      4.5          for word in words:
    10   6792010    4864567.0      0.7      6.6              if word not in word_reviews:
    11     93059      78525.0      0.8      0.1                  word_reviews[word] = []
    12   6792010    5730087.0      0.8      7.8              word_reviews[word].append(recipe_id)
    13                                           
    14         1          2.0      2.0      0.0      word_reviews_count = {}
    15    126680   18817482.0    148.5     25.5      for _, row in df.dropna(subset=["review"]).iterrows():
    16    126679    2289255.0     18.1      3.1          review = row["review"]
    17    126679    2206626.0     17.4      3.0          words = re.sub("[^A-Za-z\s]", "", review).split(" ")
    18   6918689    3513912.0      0.5      4.8          for word in words:
    19   6792010    7285732.0      1.1      9.9              word_reviews_count[word] = len(word_reviews[word])
    20         1          4.0      4.0      0.0      return word_reviews_count

на два существующих цикла тратится чуть больше 50% времени => циклы необходимо исключить

In [185]:
%time get_word_reviews_count(reviews)

CPU times: user 27.8 s, sys: 229 ms, total: 28.1 s
Wall time: 28.2 s


{'Last': 97,
 'week': 1509,
 'whole': 5782,
 'sides': 458,
 'of': 109239,
 'frozen': 2943,
 'salmon': 1032,
 'fillet': 92,
 'was': 89617,
 'on': 36121,
 'sale': 262,
 'in': 62796,
 'my': 44644,
 'local': 572,
 'supermarket': 95,
 'so': 46848,
 'I': 288243,
 'bought': 1514,
 'tons': 163,
 'okay': 600,
 'only': 14209,
 '': 330966,
 'but': 43035,
 'total': 512,
 'weight': 216,
 'over': 9801,
 'pounds': 291,
 'This': 39555,
 'recipe': 71751,
 'is': 56519,
 'perfect': 7807,
 'for': 123162,
 'even': 8074,
 'though': 4942,
 'it': 133219,
 'calls': 525,
 'steaks': 511,
 'cut': 6848,
 'up': 16178,
 'the': 266530,
 'into': 7085,
 'individual': 314,
 'portions': 214,
 'and': 219215,
 'followed': 4880,
 'instructions': 1007,
 'exactly': 4625,
 'Im': 8330,
 'one': 17615,
 'those': 2419,
 'food': 3498,
 'combining': 78,
 'diets': 45,
 'left': 5064,
 'out': 25697,
 'white': 3690,
 'wine': 1760,
 'added': 22099,
 'just': 25271,
 'a': 166467,
 'dash': 547,
 'vinegar': 1946,
 'instead': 11644,
 'little'

2.2  Оптимизируйте функцию и добейтесь значительного (как минимум, в 5 раз) прироста в скорости выполнения. Для демонстрации результата измерьте скорость выполнения оригинальной функции и функции, написанной вами.

In [52]:
from collections import Counter

In [62]:
sublist = reviews['review'].values.tolist()
ll = str(sublist).split(' ')

In [68]:



def get_word_reviews_count_optimaze(df):
    cnt = Counter(df[1:])

    

    return cnt

In [69]:
%time get_word_reviews_count_optimaze(ll)

CPU times: total: 516 ms
Wall time: 516 ms


Counter({'week': 801,
         'whole': 5627,
         'sides': 312,
         'of': 109028,
         'frozen': 2647,
         'salmon': 728,
         'fillet': 60,
         'was': 88760,
         'on': 34571,
         'sale': 147,
         'in': 61536,
         'my': 44038,
         'local': 561,
         'supermarket,': 10,
         'so': 46034,
         'I': 261303,
         'bought': 1368,
         'tons': 161,
         '(okay,': 5,
         'only': 13963,
         '3,': 48,
         'but': 42513,
         'total': 380,
         'weight': 160,
         'over': 9060,
         '10': 2258,
         'pounds).': 2,
         '': 210161,
         'This': 15894,
         'recipe': 40697,
         'is': 55070,
         'perfect': 4360,
         'for': 121211,
         'fillet,': 14,
         'even': 7877,
         'though': 2295,
         'it': 110962,
         'calls': 520,
         'steaks.': 87,
         'cut': 6687,
         'up': 13573,
         'the': 265965,
         'into': 7030,
   

## Numba

В файле `rating_predictions.json` хранятся данные о рейтингах рецептов и прогнозных значениях рейтингов для этого рецепта, полученных при помощи модели машинного обучения. 

Напишите несколько версий функции (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от прогнозного значения рейтинга для данного рецепта. 


Замечание 1: в формуле MAPE под $A_t$ понимается рейтинг из отзыва $t$, под $F_t$ - прогнозное значения рейтинга отзыва $t$.

Замечание 2: в результате работы функций должно получиться одно число - MAPE для всего набора данных.

In [71]:
import json
with open('rating_predictions.json', 'r', encoding='utf-8') as f: 
  text = json.load(f)

In [72]:
from sklearn.metrics import mean_absolute_percentage_error

In [73]:
def first_way(text):
  true = []
  predict = []
  for i in range(len(text)):
    true.append(text[i]['rating'])
    predict.append(text[i]['prediction'])
  return mean_absolute_percentage_error(true, predict)

 


In [74]:
%time first_way(text)

CPU times: total: 15.6 ms
Wall time: 34.4 ms


0.13325265503991449

3\.1 Создайте два списка `A_list` и `F_list` на основе файла `rating_predictions.json`. Напишите функцию `mape_lists` без использования векторизованных операций и методов массивов `numpy` и без использования `numba` (проитерируйтесь по спискам и вычислите суммарное значение MAPE для всех элементов, а потом усредните результат).

Измерьте время выполнения данной функции на входных данных `A_list` и `F_list`. Временем, затрачиваемым на создание списков, можно пренебречь.
    

In [75]:
A_list = []
F_list = []
for i in range(len(text)):
  A_list.append(text[i]['rating'])
  F_list.append(text[i]['prediction'])


In [76]:
! pip install --user numpy==1.20

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.


In [77]:
def mape_lists(A_list, F_list):
  abs_ = 0
  for i in range(len(A_list)):
    abs_ += abs((A_list[i] - F_list[i])/A_list[i])
  # print(1/len(A_list)*abs_)
  return 1/len(A_list)*abs_ * 100

In [78]:
%time print(f'MAPE {mape_lists(A_list, F_list)}')


MAPE 13.325265503992636
CPU times: total: 15.6 ms
Wall time: 14 ms


3\.2. Создайте массивы `numpy` `A_array` и `F_array` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numpy` с использованием векторизованных операций и методов массивов `numpy`.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`. Временем, затрачиваемым на создание массивов, можно пренебречь.

In [79]:
import numpy as np 

In [80]:
A_array = np.asarray(A_list)
F_array = np.asarray(F_list)

https://stackoverflow.com/questions/47648133/mape-calculation-in-python

In [81]:
def mape_numpy(A_array, F_array):
  mask = A_array != 0
  return (np.fabs(A_array - F_array)/A_array)[mask].mean() * 100


In [82]:
%time mape_numpy(A_array, F_array)

CPU times: total: 15.6 ms
Wall time: 1 ms


13.32526550399145

In [83]:
def mape_numpy(A_array, F_array):
  return np.mean(np.abs((A_array - F_array) / A_array)) * 100

In [84]:
%time mape_numpy(A_array, F_array)

CPU times: total: 0 ns
Wall time: 1 ms


13.32526550399145

3\.3. Создайте объекты `numba.typed.List` `A_typed` и `F_typed` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numba` без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`. 

Измерьте время выполнения данной функции на входных данных `A_typed` и `F_typed`. Временем, затрачиваемым на создание объектов `numba.typed.List`, можно пренебречь.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`.

In [85]:
A_typed = numba.typed.List(A_list)
F_typed = numba.typed.List(F_list)

In [86]:
@njit 
def mape_numba(A_list, F_list):
  abs_ = 0
  for i in range(len(A_list)):
    abs_ += abs((A_list[i] - F_list[i])/A_list[i])
  # print(1/len(A_list)*abs_)
  return 1/len(A_list)*abs_ * 100

In [90]:
%time mape_numba(A_typed, F_typed)

CPU times: total: 0 ns
Wall time: 0 ns


13.325265503992636

In [97]:
%time mape_numba(A_array, F_array)

CPU times: total: 0 ns
Wall time: 0 ns


13.325265503992636

## Векторизация

Сайт-агрегатор устроил акцию: он дарит купоны на посещение ресторана тем пользователям, оставившим отзывы, идентификатор которых является _красивым числом_. Натуральное число называется _красивым_, если первая цифра числа совпадает с последней цифрой числа. 



4\.1 Напишите функцию `is_pretty`, которая для каждого идентификатора пользователя из файла определяет, получит ли он подарок. Запрещается преобразовывать идентификатор пользователя к строке. Подтвердите корректность реализации, продемонстрировав примеры.

In [98]:
ids = reviews["recipe_id"].values
ids

array([ 57993, 142201, 252013, ..., 222001, 354979, 415599], dtype=int64)

In [181]:
ids[4091] // 10

17007

https://stackoverflow.com/questions/41271299/how-can-i-get-the-first-two-digits-of-a-number


In [241]:
def is_pretty(n: int) -> bool:
        return n%10 == n // 10 ** (int(math.log(n, 10)) - 1 + 1)
    

In [242]:
ids[5], ids[4091]

(31322, 170071)

In [243]:
is_pretty(ids[5]), is_pretty(ids[4091])

(False, True)

4\.2 Посчитайте с помощью функции `is_pretty` количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.

Привожу плохую функцию

In [247]:
def is_pretty(ids):
    cnt = 0
    for n in ids:
        if n%10 == n // 10 ** ((np.log(n)) - 1 + 1):
            cnt+=1 
    return cnt
            

In [248]:
%time is_pretty(ids)

CPU times: total: 562 ms
Wall time: 556 ms


12081

4\.3. При помощи `numpy` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [249]:
def is_pretty(ids):
    return len(ids[np.where((ids%10 == (ids // 10 ** ((np.log(ids)) - 1 + 1))))])

In [252]:
%time is_pretty(ids)

CPU times: total: 15.6 ms
Wall time: 6.96 ms


12081

4\.4. При помощи `numba` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [253]:
@njit
def is_pretty(ids):
    return len(ids[np.where((ids%10 == (ids // 10 ** ((np.log(ids)) - 1 + 1))))])


In [259]:
%time is_pretty(ids)

CPU times: total: 0 ns
Wall time: 1.96 ms


12081